# Exploration et Entraînement du Modèle de Risque de Crédit

## Objectif
Ce notebook a pour but d'analyser les données, expérimenter le pipeline de Machine Learning, entraîner le modèle final et visualiser les résultats.

Nous allons suivre les étapes suivantes :
1. Configuration et imports
2. Chargement des données
3. Analyse exploratoire (EDA)
4. Prétraitement et ingénierie des caractéristiques
5. Entraînement du modèle
6. Évaluation du modèle
7. Sauvegarde du pipeline
8. Conclusion

## 1. Configuration et Imports

Importons les bibliothèques nécessaires et configurons l'environnement de travail.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay

# Import des modules locaux
from src.model_training import load_data, build_full_pipeline, train_model, evaluate_model, save_pipeline
from src.preprocessing import NUMERICAL_COLS

# Configuration de l'affichage
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn')
%matplotlib inline

# Chemins des fichiers
DATA_FILEPATH = 'data/generated_credit_data.csv'
MODEL_DIR = 'models'
PIPELINE_FILEPATH = f'{MODEL_DIR}/full_pipeline.joblib'

## 2. Chargement des Données

Chargeons les données générées et vérifions leur structure.

In [ ]:
# Charger les données
data = load_data(DATA_FILEPATH)

if data is not None:
    print("\nAperçu des données :")
    display(data.head())
    
    print("\nInformations sur le dataset :")
    display(data.info())

## 3. Analyse Exploratoire des Données (EDA)

### 3.1 Statistiques descriptives

In [ ]:
# Statistiques descriptives
print("Statistiques descriptives :")
display(data.describe())

# Distribution de la variable cible
plt.figure(figsize=(8, 5))
sns.countplot(data=data, x='default')
plt.title('Distribution de la Variable Cible (default)')
plt.show()

# Pourcentage de chaque classe
default_dist = data['default'].value_counts(normalize=True) * 100
print("\nDistribution des classes (%):\n", default_dist)

### 3.2 Analyse des valeurs manquantes

In [ ]:
# Analyse des valeurs manquantes
missing_values = data.isnull().sum()
missing_percentages = (missing_values / len(data)) * 100

missing_df = pd.DataFrame({
    'Valeurs manquantes': missing_values,
    'Pourcentage (%)': missing_percentages
})

print("Analyse des valeurs manquantes :")
display(missing_df)

# Visualisation des valeurs manquantes
plt.figure(figsize=(10, 6))
sns.heatmap(data.isnull(), yticklabels=False, cmap='viridis')
plt.title('Heatmap des Valeurs Manquantes')
plt.show()

### 3.3 Distribution des variables numériques

In [ ]:
# Distribution des variables numériques
fig, axes = plt.subplots(3, 2, figsize=(15, 18))
axes = axes.ravel()

for idx, col in enumerate(NUMERICAL_COLS):
    sns.histplot(data=data, x=col, ax=axes[idx])
    axes[idx].set_title(f'Distribution de {col}')

plt.tight_layout()
plt.show()

# Boxplots pour détecter les outliers
fig, axes = plt.subplots(3, 2, figsize=(15, 18))
axes = axes.ravel()

for idx, col in enumerate(NUMERICAL_COLS):
    sns.boxplot(data=data, y=col, ax=axes[idx])
    axes[idx].set_title(f'Boxplot de {col}')

plt.tight_layout()
plt.show()

### 3.4 Analyse des corrélations

In [ ]:
# Matrice de corrélation
plt.figure(figsize=(10, 8))
correlation_matrix = data.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Matrice de Corrélation')
plt.show()

# Relations entre caractéristiques et variable cible
fig, axes = plt.subplots(3, 2, figsize=(15, 18))
axes = axes.ravel()

for idx, col in enumerate(NUMERICAL_COLS):
    sns.boxplot(data=data, x='default', y=col, ax=axes[idx])
    axes[idx].set_title(f'{col} vs Default')

plt.tight_layout()
plt.show()

## 4. Prétraitement et Ingénierie des Caractéristiques

Préparation des données pour l'entraînement du modèle.

In [ ]:
# Séparation des données
X = data.drop('default', axis=1)
y = data['default']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

print(f"Taille de l'ensemble d'entraînement: {X_train.shape}")
print(f"Taille de l'ensemble de test: {X_test.shape}")

# Construction du pipeline
full_pipeline = build_full_pipeline()
print("\nPipeline construit:")
print(full_pipeline)

## 5. Entraînement du Modèle

Entraînement du pipeline complet avec GridSearchCV.

In [ ]:
# Entraînement du modèle
trained_pipeline = train_model(full_pipeline, X_train, y_train)

if trained_pipeline is not None:
    print("\nMeilleurs paramètres trouvés :")
    print(trained_pipeline.get_params())

## 6. Évaluation du Modèle

Évaluation des performances sur l'ensemble de test.

In [ ]:
if trained_pipeline is not None:
    # Évaluation sur l'ensemble de test
    evaluation_results = evaluate_model(trained_pipeline, X_test, y_test)
    
    if evaluation_results is not None:
        print("\nRésultats de l'évaluation sur l'ensemble de test :")
        for metric, value in evaluation_results.items():
            if metric != 'confusion_matrix':
                print(f"{metric}: {value:.4f}")
        
        # Visualisation de la matrice de confusion
        plt.figure(figsize=(8, 6))
        cm = np.array(evaluation_results['confusion_matrix'])
        disp = ConfusionMatrixDisplay(confusion_matrix=cm)
        disp.plot()
        plt.title('Matrice de Confusion')
        plt.show()
        
        # Courbe ROC
        y_pred_proba = trained_pipeline.predict_proba(X_test)[:, 1]
        fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
        roc_auc = auc(fpr, tpr)

        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('Taux de Faux Positifs')
        plt.ylabel('Taux de Vrais Positifs')
        plt.title('Courbe ROC')
        plt.legend(loc="lower right")
        plt.show()

## 7. Sauvegarde du Pipeline

Sauvegarde du pipeline entraîné pour une utilisation future.

In [ ]:
if trained_pipeline is not None:
    save_pipeline(trained_pipeline)

## 8. Conclusion et Perspectives

### Résumé des Résultats
- Le modèle a été entraîné avec succès sur les données synthétiques.
- Les performances principales sont mesurées par l'AUC-ROC, la précision et le rappel.
- Le pipeline complet gère automatiquement le prétraitement et l'ingénierie des caractéristiques.

### Limitations
1. Données synthétiques : Le modèle est entraîné sur des données générées, qui peuvent ne pas refléter la complexité du monde réel.
2. Déséquilibre de classe : La gestion du déséquilibre est simplifiée.
3. Validation : Une validation croisée plus robuste pourrait être nécessaire.

### Améliorations Futures
1. Tester d'autres algorithmes (ex: LightGBM, XGBoost)
2. Implémenter des techniques de gestion du déséquilibre (SMOTE, class_weight)
3. Ajouter plus de caractéristiques et interactions
4. Optimiser l'hyperparamétrage avec une grille plus large
5. Mettre en place une validation plus robuste